In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import logging
import torch
from torchsummary import summary

logging.basicConfig(level=logging.INFO)

from src.data_loader.data_loader import DataLoader
from src.model.model import TwoHeadConvNeXtV2
from src.config.configuration import CLASS_NUM, AUGMENTATION_STRENGTH, NUM_AUGMENTATIONS
from src.model.utils import train_model
from src.data_loader.augmentation import Augmentor

INFO:numexpr.utils:NumExpr defaulting to 16 threads.
d:\AMM_HW\cv-amm-hw\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'AUGMENTATION_STRENGTH' from 'src.config.configuration' (d:\AMM_HW\cv-amm-hw\src\config\configuration.py)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
logging.info(f"Using device: {device}")
! nvidia-smi

In [ ]:
img_path = "data/train_images_small"
label_path = "data/venomous_status_metadata.csv"
meta_data_path = "data/train_images_metadata.csv"

data_loader = DataLoader(
    image_data_set_path=img_path,
    meta_data_path=meta_data_path,
    label_info_path=label_path
    )

In [ ]:
model = TwoHeadConvNeXtV2(num_multi_classes=CLASS_NUM)

In [ ]:
model.backbone.default_cfg

In [ ]:
summary(model, input_size=(3, 224, 224), device=str(device))

In [ ]:
augmentation_strength = AUGMENTATION_STRENGTH
center_n_transforms = augmentation_strength
center_magnitude = augmentation_strength * 5
augmentor = Augmentor(num_augmentations=NUM_AUGMENTATIONS, center_n_transforms=center_n_transforms, center_magnitude=center_magnitude)

In [ ]:
train_model(data_loader, model, augmentor=augmentor)